In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('rte500.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)



C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.head()

,Unnamed: 0,sentence1,sentence2,label
0,485,"Rodriguez, a four-year NYPD officer, suffered ...",Rodriguez died of burns to his legs.,False
1,757,North Korean refugees had been gathering in so...,Ho Chi Minh City is now called Saigon.,False
2,438,These parties are the True Path Party (rightis...,"The True Path Party, the Virtue Party and the ...",False
3,566,Lyon is actually the gastronomic capital of Fr...,Lyon is the capital of France.,False
4,52,Prime Minister Allawi has sent out letters to ...,Prime Minister Allawi will return to Iraq in t...,False


In [4]:
df1.label[df1.label == 1]="TRUE"
df1.label[df1.label == 0]="FALSE"


C:\Users\moshi\AppData\Local\Temp/ipykernel_6024/1931725721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 1]="TRUE"
C:\Users\moshi\AppData\Local\Temp/ipykernel_6024/1931725721.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 0]="FALSE"


In [5]:
df1["label"].value_counts()

FALSE    250
TRUE     250
Name: label, dtype: int64

In [6]:
temp = df1

In [7]:
text1 = temp['sentence1']+temp['sentence2']
emotion= temp['label'].tolist()

In [8]:
text=text1.values.tolist()

In [9]:
temp["label"].value_counts()

FALSE    250
TRUE     250
Name: label, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [11]:
import ktrain
from ktrain import text
MODEL_NAME = 'prajjwal1/bert-tiny'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [12]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 35
	95percentile : 63
	99percentile : 82


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 37
	95percentile : 61
	99percentile : 70


In [13]:
model = t.get_classifier()

C:\Users\moshi\anaconda3\lib\site-packages\ktrain\text\preprocessor.py:1069: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


In [14]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [15]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 29s 333ms/step - loss: 0.6945 - accuracy: 0.4925 - val_loss: 0.6877 - val_accuracy: 0.5700
Epoch 2/5
67/67 [==============================] - 21s 306ms/step - loss: 0.6934 - accuracy: 0.5100 - val_loss: 0.6859 - val_accuracy: 0.5700
Epoch 3/5
67/67 [==============================] - 22s 321ms/step - loss: 0.6898 - accuracy: 0.5725 - val_loss: 0.6832 - val_accuracy: 0.5700
Epoch 4/5
67/67 [==============================] - 22s 324ms/step - loss: 0.6868 - accuracy: 0.5525 - val_loss: 0.6801 - val_accuracy: 0.5600
Epoch 5/5
67/67 [==============================] - 22s 322ms/step - loss: 0.6786 - accuracy: 0.6000 - val_loss: 0.6798 - val_accuracy: 0.5700


In [16]:
class_names=["TRUE","FALSE"]

In [17]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        TRUE       0.53      0.52      0.53        46
       FALSE       0.60      0.61      0.61        54

    accuracy                           0.57       100
   macro avg       0.57      0.57      0.57       100
weighted avg       0.57      0.57      0.57       100



array([[24, 22],
       [21, 33]], dtype=int64)